# debug

In [1]:
%load_ext autoreload

In [2]:
%autoreload
import pathlib
import random
import numpy as np
import torch
from options.train_options import TrainOptions
from data.abImageDataset import ABImageDataset
from torch.utils.data import DataLoader
from utils.training import Trainer, Updater
from models.pix2pix_model import Pix2PixModel

import matplotlib.pyplot as plt

In [3]:
%run debug.py -n 1 -s 1 --epoch 10 --name edges2shoes --batch_size 1 --normA --normB --preprocess none --dataroot /mnt/data/pix2pix/edges2shoes/dataframe --save_dir /mnt/data/pix2pix/result_dir/

init checkpoints_dir: /mnt/hdd/data/pix2pix/result_dir/edges2shoes/result_1/result_1_1
----------------- Options ---------------
                     A_nc: 3                             
                     B_nc: 3                             
               batch_size: 1                             	[default: 4]
                    beta1: 0.5                           
          checkpoints_dir: /mnt/hdd/data/pix2pix/result_dir/edges2shoes/result_1/result_1_1	[default: None]
                coeff4dis: 1.0                           
                 cropSize: 256                           
                 dataroot: /mnt/data/pix2pix/edges2shoes/dataframe	[default: None]
                   device: cuda:0                        	[default: None]
                    epoch: 10                            	[default: 15]
                   flip_p: False                         
                 gan_mode: vanilla                       
                   gpu_id: 0                             

In [4]:
# seedの固定
random.seed(opt.seed)
np.random.seed(opt.seed)
torch.manual_seed(opt.seed)
if opt.device != torch.device("cpu"):
    torch.cuda.manual_seed(opt.seed)

In [5]:
# データ
# data path
dataroot = pathlib.Path(opt.dataroot)
trainData_path = (dataroot / "train_df.csv").resolve()
valData_path = (dataroot / "val_df.csv").resolve()
print("Train Dataset Path :", trainData_path)
print("Val Dataset Path :", valData_path)

Train Dataset Path : /mnt/hdd/data/pix2pix/edges2shoes/dataframe/train_df.csv
Val Dataset Path : /mnt/hdd/data/pix2pix/edges2shoes/dataframe/val_df.csv


In [6]:
# dataset
tmp = opt.vis_num
opt.vis_num = opt.batch_size * 10
train_dataset = ABImageDataset(df_path=trainData_path, opt=opt, vis=True)
opt.vis_num = tmp
train4vis_dataset = ABImageDataset(
    df_path=trainData_path, opt=opt, mode_inference=True, vis=True
)
val4vis_dataset = ABImageDataset(
    df_path=valData_path, opt=opt, mode_inference=True, vis=True
)
print("Train Dataset Size : {}".format(len(train_dataset)))

Train Dataset Size : 10


In [7]:
# dataloader
# dataloader for leaning
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=opt.batch_size,
    shuffle=False, # for debug
    num_workers=opt.nThreads,
)

# dataloader for visuzlizing
train4vis_dataloader = DataLoader(
    dataset=train4vis_dataset,
    batch_size=len(train4vis_dataset),
    shuffle=False,
    num_workers=opt.nThreads,
)
val4vis_dataloader = DataLoader(
    dataset=val4vis_dataset,
    batch_size=len(train4vis_dataset),
    shuffle=False,
    num_workers=opt.nThreads,
)
dataloaders4vis = {"train": train4vis_dataloader, "val": val4vis_dataloader}

In [8]:
# data の確認
data_dict = next(iter(train_dataloader))

In [ ]:
# 本物画像の保存
import math
import torchvision
plt.style.use("seaborn-white")
phase = "train"
for phase in ["train", "val"]:
    data_dict = next(iter(dataloaders4vis[phase]))
    total = opt.vis_num
    ncol = int(math.sqrt(total))
    nrow = math.ceil(float(total) / ncol)
    A = torchvision.utils.make_grid(data_dict["A"],
                                    normalize=False,
                                    nrow=nrow,
                                    padding=0)
    A = A * opt.stdA + opt.meanA
    B = torchvision.utils.make_grid(data_dict["B"],
                                    normalize=False,
                                    nrow=nrow,
                                    padding=0)
    B = B * opt.stdB + opt.meanB
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].imshow(A.permute(1, 2, 0))
    ax[0].set_axis_off()
    ax[1].imshow(B.permute(1, 2, 0))
    ax[1].set_axis_off()
    fig.subplots_adjust(wspace=0, hspace=0)
    fig.savefig(opt.image_dir / "{}_example.png".format(phase),
                 bbox_inches="tight",
                 pad_inches=0)

In [ ]:
train_dataset.transformer

In [ ]:
train4vis_dataset.transformer

In [ ]:
val4vis_dataset.transformer

In [ ]:
plt.style.use("seaborn-whitegrid")
data_dict = next(iter(train_dataloader))
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
A = data_dict["A"][0].permute(1, 2, 0) * opt.stdA + opt.meanA
B = data_dict["B"][0].permute(1, 2, 0) * opt.stdB + opt.meanB
ax[0].imshow(A)
ax[1].imshow(B)

In [ ]:
plt.style.use("seaborn-whitegrid")
data_dict = next(iter(dataloaders4vis["train"]))
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
A = data_dict["A"][0].permute(1, 2, 0) * opt.stdA + opt.meanA
B = data_dict["B"][0].permute(1, 2, 0) * opt.stdB + opt.meanB
ax[0].imshow(A)
ax[1].imshow(B)

In [ ]:
plt.style.use("seaborn-whitegrid")
data_dict = next(iter(dataloaders4vis["val"]))
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
A = data_dict["A"][0].permute(1, 2, 0) * opt.stdA + opt.meanA
B = data_dict["B"][0].permute(1, 2, 0) * opt.stdB + opt.meanB
ax[0].imshow(A)
ax[1].imshow(B)

In [ ]:
# model 構築
model = Pix2PixModel(opt)

In [ ]:
# updater
updater = Updater(train_dataloader=train_dataloader, model=model, opt=opt)

In [ ]:
# trainer
trainer = Trainer(updater, opt, dataloaders4vis)

In [ ]:
# run
trainer.run()